In [222]:
from pandas.io.json import json_normalize
from pymongo import MongoClient
import pandas as pd
import requests
from cleaning import moneyRaise, officeToGeoPoint

In [223]:
client = MongoClient("mongodb://localhost/projectdb")
db = client.get_database()

In [224]:
all_offices = list(db.companies.aggregate([{"$unwind":"$offices"},{"$project":{"offices":1,"name":1,"category_code":1, "total_money_raised":1}}]))

In [225]:
df = pd.DataFrame(all_offices)

In [226]:
print(set(df["category_code"]))

{'sports', 'other', 'real_estate', None, 'local', 'consulting', 'medical', 'games_video', 'network_hosting', 'legal', 'nanotech', 'finance', 'analytics', 'photo_video', 'search', 'news', 'health', 'fashion', 'social', 'advertising', 'biotech', 'manufacturing', 'enterprise', 'hardware', 'nonprofit', 'transportation', 'mobile', 'travel', 'design', 'web', 'automotive', 'software', 'music', 'messaging', 'security', 'government', 'semiconductor', 'cleantech', 'public_relations', 'ecommerce', 'education', 'hospitality'}


In [227]:
tech = ["analytics", "web", "cleantech", "software", "mobile", "social", "ecommerce", "search", "hardware", "biotech", "nanotech", "games_video"]
df["category_code"] = df["category_code"].apply(lambda x: "tech" if x in tech else "other" )

In [228]:
df["total_money_raised"] = df["total_money_raised"].apply(moneyRaise)

In [229]:
df = df.drop(df[df["total_money_raised"]<1000000].index)

In [230]:
cleaned_offices = df.apply(officeToGeoPoint,axis=1, result_type="expand")
cleaned_offices.columns = ["geolocation","clean_state"]

In [231]:
df = pd.concat([df,cleaned_offices], axis=1)

In [233]:
companies_clean = df[["name","category_code","total_money_raised","geolocation","clean_state"]]

In [234]:
companies_clean = companies_clean.drop(companies_clean[companies_clean["clean_state"]!="success"].index)

In [235]:
companies_clean = companies_clean.drop(companies_clean[companies_clean["category_code"]!="tech"].index)

In [236]:
companies_clean = companies_clean.sort_values(["total_money_raised"], ascending=False)

In [240]:
companies_clean.to_json("../output/companies_clean.json",orient="records")

In [238]:
companies_clean

,name,category_code,total_money_raised,geolocation,clean_state
8789,Clearwire,tech,570000000000,"{'type': 'Point', 'coordinates': [-86.9843446,...",success
5,Facebook,tech,243000000000,"{'type': 'Point', 'coordinates': [-122.151801,...",success
6,Facebook,tech,243000000000,"{'type': 'Point', 'coordinates': [-6.267494, 5...",success
7,Facebook,tech,243000000000,"{'type': 'Point', 'coordinates': [-73.9792469,...",success
15026,Terra-Gen Power,tech,120000000000,"{'type': 'Point', 'coordinates': [-73.9788753,...",success
...,...,...,...,...,...
14420,Etix,tech,1000000,"{'type': 'Point', 'coordinates': [-78.8139, 35...",success
214,Dogster,tech,1000000,"{'type': 'Point', 'coordinates': [-122.400909,...",success
14377,BCB Medical,tech,1000000,"{'type': 'Point', 'coordinates': [22.2930547, ...",success
10191,MyOtherDrive,tech,1000000,"{'type': 'Point', 'coordinates': [-84.1949463,...",success
